## **Serial Dilution then Agar Spot**

#### Deck Layout:
1. TipBox.200uL.Corning-4864.orangebox
2. Empty
3. Reservoir.12col.Agilent-201256-100.BATSgroup *(media in columns 1 and 2, refill between each run)*
4. Empty
5. Plate.96.Corning-3635.ClearUVAssay *(source plate, source for each half plate in adjacent columns)*
6. Plate.96.Corning-3635.ClearUVAssay  *(dilution plate, empty at start)*
7. TipBox.50uL.Axygen-EV-50-R-S.tealbox
8. AgarPlate.40mL.OmniTray-242811.ColonyPicker 

#### Steps:
**1. Dispense Diluent into Whole Plate** --> dilute_then_spot_STEP1.hso
* Place media in columns 1 and 2 of 96 deep well or 12 channel reservoir, be sure to refill this media between runs
       
**2. Complete All Serial Dilutions** --> dilute_then_spot_STEP2.hso
* Stock for first half of plate comes from column specified in 'stock_start_column' variable below. 
* Stock for second half of plate comes from column ('stock_start_column'+1)
* Serial Dilutions are 10 fold dilutions over 6 columns. NOTE: First column is pure undiluted stock. 

**3. Spot All Dilutions onto Agar Plate** --> dilute_then_spot_STEP3.hso
* Aspirates 10ul, dispenses 3.5uL onto agar plate at pre-calculated z-height

In [2]:
from liquidhandling import SoloSoft, SoftLinx
from liquidhandling import *  # replace with plate types used

#* Program Variables ------------------
stock_start_column = 1
num_plate_halves = 2  #(1 or 2)
spot_z_shift = 1.8

# # OR uncomment this and fill in first 2 variables to have z shift calculated for you - - - - -  - - - - - - 
# caliper_mm = 9
# measure_assist_mm = 1.6  # this is the height of the blue plastic used to aid in caliper measurement

# actual_plate_height_mm = 14.3 
# plate_height_in_db = 15
# well_depth_in_db = 10

# spot_z_shift = round((actual_plate_height_mm - (caliper_mm - measure_assist_mm)) - (plate_height_in_db - well_depth_in_db), 1)
# print(spot_z_shift)
# # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

media_aspirate_column = 1
first_column_transfer_volume = 100
dilution_media_volume = 90
dilution_transfer_volume = 10
stock_mix_volume = 50
dilution_mix_volume = 30  # 50uL tips used
num_mixes = 5

pre_spot_mix_volume = 30

default_z_shift = 2
pre_spot_aspirate_volume = 10
spot_volume = 3.5
#* ---------------------------------------
soloSoft = SoloSoft( 
    filename="modular_dilute_then_spot_STEP1.hso",
    plateList=[
        "TipBox.200uL.Corning-4864.orangebox",
        "Empty",
        "Reservoir.12col.Agilent-201256-100.BATSgroup",
        "Empty",
        "Plate.96.Corning-3635.ClearUVAssay",
        "Plate.96.Corning-3635.ClearUVAssay",
        "TipBox.50uL.Axygen-EV-50-R-S.tealbox",
        "AgarPlate.40mL.OmniTray-242811.ColonyPicker",
    ],
)

# * STEP 1: Dispense diluent into whole plate -> MUST CHECK/REFILL BETWEEN CREATING EACH PLATE -------------

# Don't dispense diluent into first column of each half plate -> will be pure stock
soloSoft.getTip()
for i in range(1,num_plate_halves + 1): 
    print("Dispensing Diluent -> plate half: " + str(i))
    for j in range((6*(i-1)+2), (6*(i-1)+7)):
        print("\tDispensing Diluent: media_column: " + str(media_aspirate_column) + "-> column: " + str(j))
        soloSoft.aspirate(
            position="Position3",
            aspirate_volumes=Reservoir_12col_Agilent_201256_100_BATSgroup().setColumn(media_aspirate_column, dilution_media_volume),
            aspirate_shift=[0, 0, 4],
        )
        soloSoft.dispense(
            position="Position6",
            dispense_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn(j, dilution_media_volume),
            dispense_shift=[0, 0, default_z_shift],
        )
    media_aspirate_column += 1
    
soloSoft.shuckTip()
soloSoft.savePipeline()

# * STEP2 SERIAL DILUTION FOR BOTH HALVES OF THE PLATE ----------------------------------------------------------------

soloSoft = SoloSoft( 
    filename="modular_dilute_then_spot_STEP2.hso",
    plateList=[
        "TipBox.200uL.Corning-4864.orangebox",
        "Empty",
        "Reservoir.12col.Agilent-201256-100.BATSgroup",
        "Empty",
        "Plate.96.Corning-3635.ClearUVAssay",
        "Plate.96.Corning-3635.ClearUVAssay",
        "TipBox.50uL.Axygen-EV-50-R-S.tealbox",
        "AgarPlate.40mL.OmniTray-242811.ColonyPicker",
    ],
)
# FIRST HALF OF THE PLATE
for i in range(1, num_plate_halves + 1):
    # set up first column of dilution plate -> pure stock, no dilution (100uL transfer volume)
    soloSoft.getTip()  # 200uL tips
    soloSoft.aspirate(
        position="Position5",
        aspirate_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn(
            stock_start_column, first_column_transfer_volume
        ),
        aspirate_shift=[0, 0, default_z_shift],
        mix_at_start=True,
        mix_volume=stock_mix_volume,
        mix_cycles=num_mixes,
        dispense_height=default_z_shift,
    )
    soloSoft.dispense(
        position="Position6",
        dispense_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn(
            (6 * (i - 1)) + 1, first_column_transfer_volume
        ),
        dispense_shift=[0, 0, default_z_shift],
        mix_at_finish=True,
        mix_volume=dilution_mix_volume,
        mix_cycles=num_mixes,
        aspirate_height=default_z_shift,
    )
    print("\nPrepare the first dilution column: ")
    print(
        "\t From clear UV column ( "
        + str(stock_start_column)
        + " ) to clear dilution UV column ( "
        + str((6 * (i - 1)) + 1)
        + " )"
    )

    print("Diluting: ")
    soloSoft.getTip("Position7") # 50uL tips for 10uL transfers
    for j in range(1,6): # 1,2,3,4,5
        soloSoft.aspirate(
            position="Position6",
            aspirate_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn(
                (6 * (i - 1)) + j, dilution_transfer_volume
            ),
            aspirate_shift=[0, 0, default_z_shift],
            mix_at_start=True,
            mix_cycles=num_mixes,
            mix_volume=dilution_mix_volume,
            dispense_height=default_z_shift,
        )
        soloSoft.dispense(
            position="Position6",
            dispense_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn(
                (6 * (i - 1)) + j + 1, dilution_transfer_volume
            ),
            dispense_shift=[0, 0, default_z_shift],
            mix_at_finish=True,
            mix_cycles=num_mixes,
            mix_volume=dilution_mix_volume,
            aspirate_height=default_z_shift,
        )

        print(
            "\t Dilute:  From clear UV column ( "
            + str((6 * (i - 1)) + j)
            + " ) to clear UV column ( "
            + str((6 * (i - 1)) + j + 1)
            + " )"
        )

    stock_start_column += 1  # make sure to draw from the next culture stock column for the next half of the plate.

soloSoft.shuckTip()
soloSoft.savePipeline()

# * STEP 3 SPOT ALL DILUTIONS -----------------------------------------------------------------------------

soloSoft = SoloSoft( 
    filename="modular_dilute_then_spot_STEP3.hso",
    plateList=[
        "TipBox.200uL.Corning-4864.orangebox",
        "Empty",
        "Reservoir.12col.Agilent-201256-100.BATSgroup",
        "Empty",
        "Plate.96.Corning-3635.ClearUVAssay",
        "Plate.96.Corning-3635.ClearUVAssay",
        "TipBox.50uL.Axygen-EV-50-R-S.tealbox",
        "AgarPlate.40mL.OmniTray-242811.ColonyPicker",
    ],
)

print("Spotting: ")
for i in range(1, (6*(num_plate_halves))+1): 
    soloSoft.getTip("Position7")
    
    soloSoft.aspirate(      # mix before aspirating the 3.5 uL 
        position="Position6", 
        aspirate_volumes=Plate_96_Corning_3635_ClearUVAssay().setColumn(i, pre_spot_aspirate_volume), 
        aspirate_shift=[0,0,default_z_shift], 
        mix_at_start=True, 
        mix_volume=pre_spot_mix_volume, 
        dispense_height=default_z_shift, 
        mix_cycles=num_mixes, 
    )
    soloSoft.dispense(
        position="Position8",
        dispense_volumes=AgarPlate_40mL_OmniTray_242811_ColonyPicker().setColumn(
            i, spot_volume
        ),
        dispense_shift=[0, 0, spot_z_shift],
    )
    print(
        "Dilution Plate Column ( "
        + str(i)
        + " ) -> Agar Plate Column ( "
        + str(i)
        + " )"
    )

soloSoft.shuckTip()
soloSoft.savePipeline()

# LOAD PROTOCOL STEPS 1-3 IN SOFTLINX
softLinx = SoftLinx("Modular Dilute then Spot Steps 1-3", "modular_dilute_then_spot.slvp")
softLinx.soloSoftRun("C:\\Users\\svcaibio\\Dev\\liquidhandling\\example\\other_protocols\\dilute_then_spot\\modular_dilute_then_spot_STEP1.hso")
softLinx.soloSoftRun("C:\\Users\\svcaibio\\Dev\\liquidhandling\\example\\other_protocols\\dilute_then_spot\\modular_dilute_then_spot_STEP2.hso")
softLinx.soloSoftRun("C:\\Users\\svcaibio\\Dev\\liquidhandling\\example\\other_protocols\\dilute_then_spot\\modular_dilute_then_spot_STEP3.hso")
softLinx.saveProtocol()

Dispensing Diluent -> plate half: 1
	Dispensing Diluent: media_column: 1-> column: 2
	Dispensing Diluent: media_column: 1-> column: 3
	Dispensing Diluent: media_column: 1-> column: 4
	Dispensing Diluent: media_column: 1-> column: 5
	Dispensing Diluent: media_column: 1-> column: 6
Dispensing Diluent -> plate half: 2
	Dispensing Diluent: media_column: 2-> column: 8
	Dispensing Diluent: media_column: 2-> column: 9
	Dispensing Diluent: media_column: 2-> column: 10
	Dispensing Diluent: media_column: 2-> column: 11
	Dispensing Diluent: media_column: 2-> column: 12

Prepare the first dilution column: 
	 From clear UV column ( 1 ) to clear dilution UV column ( 1 )
Diluting: 
	 Dilute:  From clear UV column ( 1 ) to clear UV column ( 2 )
	 Dilute:  From clear UV column ( 2 ) to clear UV column ( 3 )
	 Dilute:  From clear UV column ( 3 ) to clear UV column ( 4 )
	 Dilute:  From clear UV column ( 4 ) to clear UV column ( 5 )
	 Dilute:  From clear UV column ( 5 ) to clear UV column ( 6 )

Prepare 